In [ ]:
# Установка необходимых библиотек (выполнить один раз)
!pip install psycopg2-binary pandas matplotlib seaborn plotly

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
import psycopg2
import warnings
warnings.filterwarnings('ignore')

# Настройка отображения
plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print('✓ Библиотеки загружены успешно')

In [ ]:
# Параметры подключения к БД
DB_CONFIG = {
    'host': 'postgres',  # Имя сервиса в Docker
    'port': 5432,
    'dbname': 'weather_db',
    'user': 'weather_user',
    'password': 'weather_pass'
}

def get_connection():
    """Создаёт подключение к базе данных"""
    return psycopg2.connect(**DB_CONFIG)

def load_data(query):
    """Загружает данные из БД в DataFrame"""
    conn = get_connection()
    df = pd.read_sql(query, conn)
    conn.close()
    return df

print('✓ Функции подключения готовы')

## 📊 Загрузка и обзор данных

In [ ]:
# Загрузка всех данных погодной станции
query = """
SELECT id, timestamp, temperature, humidity, pressure, 
       wind_speed, wind_direction, weather_condition
FROM weather_data
ORDER BY timestamp DESC
"""

df = load_data(query)

print(f'Загружено записей: {len(df)}')
print(f'Период данных: с {df["timestamp"].min()} по {df["timestamp"].max()}')
print('\nПервые 5 записей:')
df.head()

In [ ]:
# Общая статистика по числовым полям
print('📈 Статистика по показаниям погодной станции:\n')
df[['temperature', 'humidity', 'pressure', 'wind_speed']].describe().round(2)

## 🌡️ Анализ температуры

In [ ]:
# График температуры во времени
fig = px.line(df.sort_values('timestamp'), 
              x='timestamp', y='temperature',
              title='🌡️ Изменение температуры во времени',
              labels={'temperature': 'Температура (°C)', 'timestamp': 'Время'})
fig.update_layout(height=400)
fig.show()

In [ ]:
# Распределение температуры
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Гистограмма
axes[0].hist(df['temperature'], bins=30, color='coral', edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Температура (°C)')
axes[0].set_ylabel('Частота')
axes[0].set_title('Распределение температуры')
axes[0].axvline(df['temperature'].mean(), color='red', linestyle='--', label=f'Среднее: {df["temperature"].mean():.1f}°C')
axes[0].legend()

# Boxplot
axes[1].boxplot(df['temperature'], vert=True)
axes[1].set_ylabel('Температура (°C)')
axes[1].set_title('Boxplot температуры')

plt.tight_layout()
plt.show()

## 💧 Анализ влажности и давления

In [ ]:
# Корреляция между параметрами
correlation_matrix = df[['temperature', 'humidity', 'pressure', 'wind_speed']].corr()

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            xticklabels=['Температура', 'Влажность', 'Давление', 'Скор. ветра'],
            yticklabels=['Температура', 'Влажность', 'Давление', 'Скор. ветра'])
ax.set_title('Корреляция между параметрами погоды')
plt.tight_layout()
plt.show()

In [ ]:
# Scatter plot: Температура vs Влажность
fig = px.scatter(df, x='temperature', y='humidity', color='weather_condition',
                 title='Зависимость влажности от температуры',
                 labels={'temperature': 'Температура (°C)', 'humidity': 'Влажность (%)'},
                 opacity=0.6)
fig.update_layout(height=500)
fig.show()

## 💨 Анализ ветра

In [ ]:
# Распределение направлений ветра (роза ветров)
wind_counts = df['wind_direction'].value_counts()

# Порядок направлений для корректного отображения
directions_order = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
wind_counts = wind_counts.reindex(directions_order).fillna(0)

fig = go.Figure()
fig.add_trace(go.Barpolar(
    r=wind_counts.values,
    theta=directions_order,
    marker_color='rgb(106, 168, 79)',
    opacity=0.8
))

fig.update_layout(
    title='🧭 Роза ветров (распределение направлений)',
    polar=dict(
        radialaxis=dict(visible=True),
        angularaxis=dict(direction='clockwise')
    ),
    height=500
)
fig.show()

In [ ]:
# Средняя скорость ветра по направлениям
wind_speed_by_direction = df.groupby('wind_direction')['wind_speed'].agg(['mean', 'max']).round(2)
wind_speed_by_direction = wind_speed_by_direction.reindex(directions_order)

print('Средняя и максимальная скорость ветра по направлениям:\n')
wind_speed_by_direction.columns = ['Средняя (м/с)', 'Максимальная (м/с)']
wind_speed_by_direction

## ☁️ Анализ погодных условий

In [ ]:
# Распределение погодных условий
weather_counts = df['weather_condition'].value_counts()

fig = px.pie(values=weather_counts.values, names=weather_counts.index,
             title='☁️ Распределение погодных условий',
             hole=0.4)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(height=500)
fig.show()

In [ ]:
# Средние показатели по типу погоды
weather_stats = df.groupby('weather_condition').agg({
    'temperature': 'mean',
    'humidity': 'mean',
    'pressure': 'mean',
    'wind_speed': 'mean'
}).round(2)

weather_stats.columns = ['Средняя температура (°C)', 'Средняя влажность (%)', 
                          'Среднее давление (гПа)', 'Средняя скорость ветра (м/с)']

print('📊 Средние показатели по типу погоды:\n')
weather_stats

## 📈 Итоговая сводка

In [ ]:
# Итоговая статистика
print('=' * 60)
print('📊 ИТОГОВАЯ СВОДКА ДАННЫХ ПОГОДНОЙ СТАНЦИИ')
print('=' * 60)
print(f'\n📅 Период наблюдений:')
print(f'   Начало: {df["timestamp"].min()}')
print(f'   Конец:  {df["timestamp"].max()}')
print(f'   Всего записей: {len(df)}')
print(f'\n🌡️ Температура:')
print(f'   Минимум: {df["temperature"].min():.1f}°C')
print(f'   Максимум: {df["temperature"].max():.1f}°C')
print(f'   Среднее: {df["temperature"].mean():.1f}°C')
print(f'\n💧 Влажность:')
print(f'   Минимум: {df["humidity"].min():.1f}%')
print(f'   Максимум: {df["humidity"].max():.1f}%')
print(f'   Среднее: {df["humidity"].mean():.1f}%')
print(f'\n📊 Давление:')
print(f'   Минимум: {df["pressure"].min():.1f} гПа')
print(f'   Максимум: {df["pressure"].max():.1f} гПа')
print(f'   Среднее: {df["pressure"].mean():.1f} гПа')
print(f'\n💨 Скорость ветра:')
print(f'   Минимум: {df["wind_speed"].min():.1f} м/с')
print(f'   Максимум: {df["wind_speed"].max():.1f} м/с')
print(f'   Среднее: {df["wind_speed"].mean():.1f} м/с')
print(f'\n☁️ Самое частое состояние погоды: {df["weather_condition"].mode()[0]}')
print('=' * 60)